### installs and imports

In [1]:
!pip install paddleocr paddlepaddle -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 

In [3]:
!pip install jiwer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.4 MB/s eta 0:00:00


In [4]:
import paddleocr
import os
import pandas as pd
import numpy as np
from pathlib import Path
from jiwer import wer, cer
from sklearn.model_selection import train_test_split
from paddleocr import PaddleOCR
from tqdm.auto import tqdm

### load trained custom model

In [5]:
custom_model_dir = "drive/MyDrive/paddleocr-files/custom_model"

In [ ]:
ocr_engine = PaddleOCR(text_recognition_model_name="en_PP-OCRv5_mobile_rec",text_recognition_model_dir=custom_model_dir)

### paths setups and gt csv loading

In [7]:
GT_CSV = Path("drive/MyDrive/paddleocr-files/full_ground_truth_lines.csv")
IMAGE_BASE_DIR = Path("drive/MyDrive/paddleocr-files/images")

In [8]:
try:
    gt_df = pd.read_csv(GT_CSV)
    print(f"Loaded {len(gt_df)} total samples from {GT_CSV}.")

    gt_df = gt_df.drop('Unnamed: 2', axis=1)
    train_df, test_df = train_test_split(gt_df, test_size=0.15, random_state=42)

    print(f"Split into {len(train_df)} training samples and {len(test_df)} testing samples.")

    if 'image_path' not in train_df.columns or 'transcription' not in train_df.columns:
        raise ValueError("CSV must contain 'image_path' and 'transcription' columns.")

    print("\nSample image path from CSV:")
    print(test_df.iloc[0]['image_path'])
    print("\nTraining DataFrame head:")
    print(test_df.head())

except FileNotFoundError:
    print(f"Error: Could not find {GT_CSV}.")
except Exception as e:
    print(f"An error occurred: {e}")

Loaded 870 total samples from drive/MyDrive/paddleocr-files/full_ground_truth_lines.csv.
Split into 739 training samples and 131 testing samples.

Sample image path from CSV:
image_splits/training_set_splits/segmented_lines_per_image/data_70/line_4.png

Training DataFrame head:
                                            image_path  \
394  image_splits/training_set_splits/segmented_lin...   
66   image_splits/training_set_splits/segmented_lin...   
495  image_splits/training_set_splits/segmented_lin...   
67   image_splits/training_set_splits/segmented_lin...   
855  image_splits/training_set_splits/synthetic_seg...   

                         transcription  
394                                 #7  
66                            (Kenzar)  
495       Sig: 1 tab 3x a day x 5 days  
67                                #100  
855  S: 1 tab every 4 hours for 8 days  


### test and show results

In [48]:
test_df = pd.read_csv("test_gt.csv")

In [50]:
test_df

,image_path,transcription
0,/content/drive/MyDrive/paddleocr-files/images/...,Amlodipine 5mg (Provasc) #100 S: 1 tablet once...
1,/content/drive/MyDrive/paddleocr-files/images/...,Rx Azithromycin 200mg/5ml syrup Sig: 3ml once ...
2,/content/drive/MyDrive/paddleocr-files/images/...,1. Vitamin B Complex (PHAREX) #200 170 S: 1 ta...
3,/content/drive/MyDrive/paddleocr-files/images/...,RX Amlopidine 5mg (Provasc) #100 S: 1 tablet o...
4,/content/drive/MyDrive/paddleocr-files/images/...,NAPHAZOLINE #2 PHENIRAMINE MALEATE (VISTALLERG...
5,/content/drive/MyDrive/paddleocr-files/images/...,Amlodpipine 5mg (Bezam / Provasc) S: 1 tablet ...
6,/content/drive/MyDrive/paddleocr-files/images/...,Medicine Frequency Breakfast Lunch 3pm Supper ...
7,/content/drive/MyDrive/paddleocr-files/images/...,Olanzapine 10 mg – # 30 Sig: 1 tablet 2x daily...
8,/content/drive/MyDrive/paddleocr-files/images/...,TRIAMCINOLONE Acetonide # 4 Sig: Apply lotion ...
9,/content/drive/MyDrive/paddleocr-files/images/...,Spironolactone 25 mg – # 8 Sig 1 tab 2x a week...


In [ ]:
results = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Running Inference"):
    gt_text = row['transcription']
    img_path = row['image_path']

    try:
        print("Processing: ", img_path)
        result = ocr_engine.predict(img_path)

        pred_text = " ".join(result[0]['rec_texts'])
        confidence = result[0]['rec_scores']

        wer_val = wer(gt_text, pred_text)
        cer_val = cer(gt_text, pred_text)

        results.append({
            "image_path": img_path,
            "gt": gt_text,
            "pred": pred_text,
            "confidence": confidence,
            "wer": wer_val,
            "cer": cer_val
        })
    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        results.append({
            "image_path": img_path,
            "gt": gt_text,
            "pred": "[ERROR]",
            "confidence": 0,
            "wer": 1.0,
            "cer": 1.0
        })

# --- 3. Save and Show Results ---
results_df = pd.DataFrame(results)
results_df.to_csv("paddle_predictions.csv", index=False)


In [57]:
print("\n--- Inference Complete ---")
print(f"Mean WER: {results_df['wer'].mean():.4f}")
print(f"Mean CER: {results_df['cer'].mean():.4f}")
acc = (1 - results_df['cer'].mean()) * 100
print(f"\nOverall Character Accuracy: {acc:.4f}%")

print("\nSample Predictions:")
print(results_df.head())


--- Inference Complete ---
Mean WER: 0.8803
Mean CER: 0.5597

Overall Character Accuracy: 44.0341%

Sample Predictions:
                                          image_path  \
0  /content/drive/MyDrive/paddleocr-files/images/...   
1  /content/drive/MyDrive/paddleocr-files/images/...   
2  /content/drive/MyDrive/paddleocr-files/images/...   
3  /content/drive/MyDrive/paddleocr-files/images/...   
4  /content/drive/MyDrive/paddleocr-files/images/...   

                                                  gt  \
0  Amlodipine 5mg (Provasc) #100 S: 1 tablet once...   
1  Rx Azithromycin 200mg/5ml syrup Sig: 3ml once ...   
2  1. Vitamin B Complex (PHAREX) #200 170 S: 1 ta...   
3  RX Amlopidine 5mg (Provasc) #100 S: 1 tablet o...   
4  NAPHAZOLINE #2 PHENIRAMINE MALEATE (VISTALLERG...   

                                                pred  \
0  ennD- Pnormc) #100 acea8 Ithhel 140 uns mutgon...   
1  olars io warter mce a dw dissolve one sacper v...   
2  O A 170 0y S 3re .8 4 ( W TH mnmoo

In [52]:
results_df

,image_path,gt,pred,confidence,wer,cer
0,/content/drive/MyDrive/paddleocr-files/images/...,Amlodipine 5mg (Provasc) #100 S: 1 tablet once...,ennD- Pnormc) #100 acea8 Ithhel 140 uns mutgon...,"[0.4147528111934662, 0.6905224919319153, 0.780...",0.903226,0.682927
1,/content/drive/MyDrive/paddleocr-files/images/...,Rx Azithromycin 200mg/5ml syrup Sig: 3ml once ...,olars io warter mce a dw dissolve one sacper v...,"[0.7436127662658691, 0.7769166231155396, 0.583...",0.964286,0.730263
2,/content/drive/MyDrive/paddleocr-files/images/...,1. Vitamin B Complex (PHAREX) #200 170 S: 1 ta...,O A 170 0y S 3re .8 4 ( W TH mnmoom Buf0 D Du2...,"[0.2099563181400299, 0.3731776773929596, 0.994...",0.978261,0.741667
3,/content/drive/MyDrive/paddleocr-files/images/...,RX Amlopidine 5mg (Provasc) #100 S: 1 tablet o...,X (Proroic) #100 :3 1 Duet mce aday boraitan s...,"[0.2640916407108307, 0.7236162424087524, 0.998...",0.774194,0.388535
4,/content/drive/MyDrive/paddleocr-files/images/...,NAPHAZOLINE #2 PHENIRAMINE MALEATE (VISTALLERG...,TIT VOCINE HENRANE MAEATE V157mERa gedi &x a d...,"[0.6561564803123474, 0.8900070786476135, 0.707...",0.931034,0.596273
5,/content/drive/MyDrive/paddleocr-files/images/...,Amlodpipine 5mg (Bezam / Provasc) S: 1 tablet ...,cuuls dgune gomn (Bezanil Prorozc) #100 day ta...,"[0.6006303429603577, 0.6523674130439758, 0.375...",0.903226,0.542683
6,/content/drive/MyDrive/paddleocr-files/images/...,Medicine Frequency Breakfast Lunch 3pm Supper ...,Frequency Breakfast Supper Bedtime Medicine Lu...,"[0.9999377131462097, 0.9998430609703064, 0.999...",0.892308,0.552326
7,/content/drive/MyDrive/paddleocr-files/images/...,Olanzapine 10 mg – # 30 Sig: 1 tablet 2x daily...,.Olanzapine 10 mg - 3D Bez0itablet 2x daily Ce...,"[0.8360777497291565, 0.7296028137207031, 0.826...",0.567568,0.213836
8,/content/drive/MyDrive/paddleocr-files/images/...,TRIAMCINOLONE Acetonide # 4 Sig: Apply lotion ...,TRIAMCINOLONE ACtonide AA Lig Apply lotion to ...,"[0.8740599155426025, 0.8714596629142761, 0.793...",0.825397,0.334395
9,/content/drive/MyDrive/paddleocr-files/images/...,Spironolactone 25 mg – # 8 Sig 1 tab 2x a week...,after breakpas Tansuloin 400 mcg - #30 Sodiun ...,"[0.9422691464424133, 0.8827236294746399, 0.769...",0.831683,0.408907


### sandbox

In [22]:
image_path = "drive/MyDrive/paddleocr-files/images/"

In [12]:
path2 = "/content/drive/MyDrive/paddleocr-files/images/image_splits/testing_set_full/data_63.png"

In [10]:
print(str(IMAGE_BASE_DIR) + "/" + str(gt_df['image_path'][0]))

drive/MyDrive/paddleocr-files/images/image_splits/training_set_splits/segmented_lines_per_image/data_1/line_0.png


In [13]:
result = ocr_engine.predict(path2)

In [23]:
path3 = image_path + "image_splits/training_set_splits/segmented_lines_per_image/data_8/line_9.png"

In [24]:
result2 = ocr_engine.predict(path3)

In [27]:
result2

[{'input_path': 'drive/MyDrive/paddleocr-files/images/image_splits/training_set_splits/segmented_lines_per_image/data_8/line_9.png',
  'page_index': None,
  'doc_preprocessor_res': {'input_path': None,
   'page_index': None,
   'input_img': array([[[167, ..., 171],
           ...,
           [174, ..., 178]],
   
          ...,
   
          [[165, ..., 169],
           ...,
           [174, ..., 178]]], dtype=uint8),
   'model_settings': {'use_doc_orientation_classify': True,
    'use_doc_unwarping': True},
   'angle': 0,
   'rot_img': array([[[167, ..., 171],
           ...,
           [174, ..., 178]],
   
          ...,
   
          [[165, ..., 169],
           ...,
           [174, ..., 178]]], dtype=uint8),
   'output_img': array([[[169, ..., 173],
           ...,
           [166, ..., 159]],
   
          ...,
   
          [[157, ..., 168],
           ...,
           [174, ..., 178]]], dtype=uint8)},
  'dt_polys': [],
  'model_settings': {'use_doc_preprocessor': True,
   'use_

In [26]:
print(" ".join(result2[0]['rec_texts']))

In [19]:
print(" ".join(result[0]['rec_texts']))
print(result[0]['rec_scores'])

Etnonicoxib 120mg1ths Ith ew lay vit B complex 5-Vibee ith emn lay 12
[0.6579744815826416, 0.7399852275848389, 0.7801142930984497, 0.7892418503761292, 0.9130263328552246, 0.6539115309715271, 0.9146913290023804]


In [28]:
import glob

In [39]:
LINES_DIR = "/content/drive/MyDrive/paddleocr-files/images/image_splits/testing_set_full"
CSV_PATH = "/content/new_gt.csv"

In [45]:
import glob
import pandas as pd
import os
import re

def get_sort_key(filepath):
    """Extracts the numerical part from filenames like 'data_11.png' for sorting."""
    filename = os.path.basename(filepath)
    # Assumes a pattern like 'data_123.png'. Adjust regex if your pattern is different.
    match = re.search(r'data_(\d+)\.png', filename)
    if match:
        return int(match.group(1))
    # Put files that don't match the pattern at the end
    return float('inf')

# --- Modified Block ---

# 1. Get all pngs (unsorted)
png_files = glob.glob(f"{LINES_DIR}/**/*.png", recursive=True)

# 2. Sort the list using the custom key function for natural sorting
pngs = sorted(png_files, key=get_sort_key)
assert len(pngs) > 0, f"No PNGs found under {LINES_DIR}. Check your paths."

# 3. Create the DataFrame
df_new = pd.DataFrame({
    "image_path": [p.replace("\\","/") for p in pngs],
    "transcription": [""]*len(pngs)
})

# 4. Filter for existing paths.
#    The DataFrame is already in the correct order, so we just reset the index.
#    The incorrect .sort_values("image_path") has been removed.
df_new = df_new[df_new["image_path"].apply(os.path.exists)].reset_index(drop=True)

# 5. Display and save
print(df_new)
df_new.to_csv(CSV_PATH, index=False)

                                           image_path transcription
0   /content/drive/MyDrive/paddleocr-files/images/...              
1   /content/drive/MyDrive/paddleocr-files/images/...              
2   /content/drive/MyDrive/paddleocr-files/images/...              
3   /content/drive/MyDrive/paddleocr-files/images/...              
4   /content/drive/MyDrive/paddleocr-files/images/...              
5   /content/drive/MyDrive/paddleocr-files/images/...              
6   /content/drive/MyDrive/paddleocr-files/images/...              
7   /content/drive/MyDrive/paddleocr-files/images/...              
8   /content/drive/MyDrive/paddleocr-files/images/...              
9   /content/drive/MyDrive/paddleocr-files/images/...              
10  /content/drive/MyDrive/paddleocr-files/images/...              
11  /content/drive/MyDrive/paddleocr-files/images/...              
12  /content/drive/MyDrive/paddleocr-files/images/...              
13  /content/drive/MyDrive/paddleocr-files/image